In [16]:
import chess # this is to handle game rules and boar represenation
import random # to generate random moves 

piece_values = { 
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330, 
    chess.ROOK: 500, 
    chess.QUEEN: 900, 
    chess.KING: 20000
}

#Defining a dictionary of piece values for material evaluation

In [17]:
#This function will assign a numerical value to a given chess position. 
#It considers material balance and a simple measure of piece mobility
#Positive scores favor white, negative favor black

def evaluate_board(board): 
    if board.is_checkmate(): 
        #If position is checkmate, return large value 
        #The player who is not in turn is the winner 
        return -10000 if board.turn else 10000

    if board.is_stalemate() or board.is_insufficient_material(): 
        #If draw return 0 
        return 0 

    #Initialize the score 
    score = 0

    #Evaluating material balance 
    for piece_type in piece_values: 
        score += len(board.pieces(piece_type, chess.WHITE)) * piece_values[piece_type]
        score += len(board.pieces(piece_type, chess.BLACK)) * piece_values[piece_type]

    #Evaluating piece mobility simplified 
    score += len(list(board.legal_moves)) * (1 if board.turn == chess.WHITE else -1)

    return score

In [18]:
"""Deploying the core of the chess engine. Implementing the Minimax Algo with Alpha-Beta 
pruning. It recursively evaluates the posisions, alternating between maximizing for white and
minimizing for black"""

def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over(): 
        return evaluate_board(board)

    if maximizing_player: 
        max_eval = float('-inf')
        for move in board.legal_moves: 
                board.push(move)
                eval = minimax(board, depth -1, alpha,beta,False)
                board.pop()
                max_eval = max(max_eval,eval)
                alpha = max(alpha, eval)
                if beta <= alpha: 
                    break 
        return max_eval

    else: 
        min_eval = float('inf')
        for move in board.legal_moves: 
                board.push(move)
                eval = minimax(board, depth -1, alpha,beta,True)
                board.pop()
                min_eval = max(min_eval,eval)
                alpha = max(alpha, eval)
                if beta <= alpha: 
                    break 
        return min_eval

In [25]:
#Finding the best move 
#Uses minimax algo to evaluate all possible moves and select best one for current player 

def get_best_move(board, depth):
    """
    Find the best move for the current position using Minimax.
    If no best move is found, return a random legal move.
    """
    best_move = None
    best_value = float('-inf') if board.turn == chess.WHITE else float('inf')
    alpha = float('-inf')
    beta = float('inf')

    for move in board.legal_moves:
        board.push(move)
        if board.turn == chess.WHITE:
            value = minimax(board, depth - 1, alpha, beta, False)
            if value > best_value:
                best_value = value
                best_move = move
            alpha = max(alpha, value)
        else:
            value = minimax(board, depth - 1, alpha, beta, True)
            if value < best_value:
                best_value = value
                best_move = move
            beta = min(beta, value)
        board.pop()

    # If no best move was found (e.g., in case of a checkmate or stalemate),
    # return a random legal move
    if best_move is None and board.legal_moves:
        best_move = random.choice(list(board.legal_moves))

    return best_move

In [26]:
#Playing the game between AI and random moves
def play_game():
    """
    Play a game between the AI and random moves.
    """
    board = chess.Board()
    
    while not board.is_game_over():
        if board.turn == chess.WHITE:
            move = get_best_move(board, depth=3)
        else:
            move = random.choice(list(board.legal_moves)) if board.legal_moves else None
        
        if move is None:
            print("No legal moves available. Game over.")
            break

        print(f"{'White' if board.turn == chess.WHITE else 'Black'} plays: {move}")
        board.push(move)
        print(board)
        print("\n")

    print("Game Over")
    print("Result:", board.result())

In [27]:
play_game()

White plays: b1a3
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
N . . . . . . .
P P P P P P P P
R . B Q K B N R


Black plays: d7d6
r n b q k b n r
p p p . p p p p
. . . p . . . .
. . . . . . . .
. . . . . . . .
N . . . . . . .
P P P P P P P P
R . B Q K B N R


White plays: f2f4
r n b q k b n r
p p p . p p p p
. . . p . . . .
. . . . . . . .
. . . . . P . .
N . . . . . . .
P P P P P . P P
R . B Q K B N R


Black plays: e7e5
r n b q k b n r
p p p . . p p p
. . . p . . . .
. . . . p . . .
. . . . . P . .
N . . . . . . .
P P P P P . P P
R . B Q K B N R


White plays: g1f3
r n b q k b n r
p p p . . p p p
. . . p . . . .
. . . . p . . .
. . . . . P . .
N . . . . N . .
P P P P P . P P
R . B Q K B . R


Black plays: a7a5
r n b q k b n r
. p p . . p p p
. . . p . . . .
p . . . p . . .
. . . . . P . .
N . . . . N . .
P P P P P . P P
R . B Q K B . R


White plays: f3h4
r n b q k b n r
. p p . . p p p
. . . p . . . .
p . . . p . . .
. . . . . P . N
N . . . . . . 